In [ ]:
import json
import networkx as nx
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from shapely.geometry import Point, LineString, MultiLineString, Polygon, box
import numpy as np
from itertools import combinations
from pyproj import CRS

from gnn_package import data_utils
from gnn_package.src.preprocessing.graph_manipulation import (
    connect_components,
    snap_points_to_network,
    explode_multilinestrings,
)
from gnn_package.src.preprocessing.graph_analysis import (
    analyze_graph_components,
    analyze_network_graph,
    validate_snapped_points,
)
from gnn_package.src.preprocessing.import_graph import get_street_network_gdfs

from gnn_package.src.preprocessing.graph_computation import (
    compute_shortest_paths,
    create_weighted_graph_from_paths,
)

from gnn_package.src.preprocessing.graph_visualization import (
    visualize_sensor_graph,
    visualize_network_components,
)

In [ ]:
# os_processed_data_dir = "../gnn_package/data/ordinance_survey/processed/"
urban_observatory_dir = "../gnn_package/data/urban_observatory/"

sensor_nodes_gdf = data_utils.read_pickled_gdf(
    urban_observatory_dir, "sensor_nodes.pkl"
)

sensor_names = json.load(open(urban_observatory_dir + "sensor_names.json"))

# trim both gdfs for testing
polygon_bbox = Polygon(
    [
        [-1.61327, 54.96188],
        [-1.59993, 54.96188],
        [-1.59993, 54.98084],
        [-1.61327, 54.98084],
    ]
)

# Create a GeoDataFrame from the bounding box polygon
bbox_gdf = gpd.GeoDataFrame(geometry=[polygon_bbox], crs="EPSG:4326")
bbox_gdf = bbox_gdf.to_crs("EPSG:27700")

sensor_nodes_gdf = gpd.clip(sensor_nodes_gdf, bbox_gdf)
# road_network_gdf = gpd.clip(road_network_gdf, bbox_gdf)

In [ ]:
def visualize_network(G, filepath=None):
    """
    Visualize the pedestrian network.
    """
    # Plot the graph
    fig, ax = ox.plot_graph(
        G,
        node_size=5,
        node_color="black",
        edge_color="#777777",
        edge_linewidth=1,
        edge_alpha=0.7,
        bgcolor="white",
        show=False,
        close=False,
    )

    if filepath:
        plt.savefig(filepath, dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# Example usage
place_name = "Newcastle upon Tyne, UK"  # Replace with your area of interest

# Get the network
network_gdf = get_street_network_gdfs(place_name)
clipped_network_gdf = gpd.clip(network_gdf, bbox_gdf)

In [ ]:
# visualize_network_components(clipped_network_gdf)

In [ ]:
snapped_sensor_nodes_gdf = snap_points_to_network(sensor_nodes_gdf, clipped_network_gdf)

In [ ]:
clipped_network_gdf = explode_multilinestrings(clipped_network_gdf)

connected_network_gdf = connect_components(clipped_network_gdf)

In [ ]:
clipped_network_gdf

In [ ]:
# Plot the gdfs
fig, ax = plt.subplots(figsize=(10, 10))
connected_network_gdf.plot(ax=ax, color="black", alpha=0.5)
snapped_sensor_nodes_gdf.plot(ax=ax, color="red", alpha=0.5)

In [ ]:
validate_snapped_points(snapped_sensor_nodes_gdf, connected_network_gdf)

In [ ]:
# compute shortest paths
shortest_paths_gdf = compute_shortest_paths(
    connected_network_gdf, snapped_sensor_nodes_gdf
)

In [ ]:
# Plot the gdfs
fig, ax = plt.subplots(figsize=(10, 10))
connected_network_gdf.plot(ax=ax, color="black", alpha=0.5)
snapped_sensor_nodes_gdf.plot(ax=ax, color="red", alpha=0.5)
shortest_paths_gdf.plot(ax=ax, color="orange", alpha=0.5)

In [ ]:
G = create_weighted_graph_from_paths(shortest_paths_gdf)

In [ ]:
visualize_sensor_graph(G, snapped_sensor_nodes_gdf)

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Get a sorted list of node IDs to ensure consistent ordering
node_ids = sorted(list(G.nodes()))

# Create the adjacency matrix using NetworkX's built-in function
adj_matrix = nx.adjacency_matrix(G, nodelist=node_ids, weight="weight")

# Convert to dense numpy array for easier viewing
adj_matrix_dense = adj_matrix.todense()

# Create a DataFrame for better visualization
adj_df = pd.DataFrame(adj_matrix_dense, index=node_ids, columns=node_ids)

print("Shape of adjacency matrix:", adj_matrix_dense.shape)
print("\nFirst few rows and columns of adjacency matrix:")
print(adj_df.iloc[:5, :5])

# Basic statistics
non_zero_weights = adj_matrix_dense[adj_matrix_dense > 0]
print("\nDistance statistics:")
print(f"Minimum distance: {np.min(non_zero_weights):.2f} meters")
print(f"Maximum distance: {np.max(non_zero_weights):.2f} meters")
print(f"Average distance: {np.mean(non_zero_weights):.2f} meters")

In [ ]:
# Create the heatmap figure
fig = go.Figure(
    data=go.Heatmap(
        z=adj_matrix_dense,
        x=node_ids,
        y=node_ids,
        colorscale="YlOrRd",
        colorbar=dict(title="Distance (meters)", thickness=15, x=1.1),
        hoverongaps=False,
        hovertemplate="From Node: %{y}<br>"
        + "To Node: %{x}<br>"
        + "Distance: %{z:.2f} meters<br>"
        + "<extra></extra>",  # This removes the secondary box in the hover
    )
)

# Update the layout
fig.update_layout(
    title="Sensor Network Adjacency Matrix",
    width=800,
    height=600,
    xaxis=dict(
        title="To Node",
        side="bottom",
        scaleanchor="y",
        scaleratio=1,
    ),
    yaxis=dict(
        title="From Node", autorange="reversed"  # This puts (0,0) at the top-left
    ),
    margin=dict(l=60, r=100, b=60, t=40, pad=4),
)

# Show the plot
fig.show()

# Optionally save as HTML
# fig.write_html("adjacency_matrix_heatmap.html")

In [ ]:
np.save("../gnn_package/data/preprocessed/test/small_adj_matrix.npy", adj_matrix_dense)
node_ids = np.array(node_ids)
np.save("../gnn_package/data/preprocessed/test/small_node_ids.npy", node_ids)